<center> <h1 style="background-color:DarkSlateBlue; color:white" >Movie Recommendation System</h1> 

![Image](https://www.vshsolutions.com/wp-content/uploads/2020/02/recommender-system-for-movie-recommendation.jpg)

<center>
<br>    
<a id="top"></a>    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:DarkSlateBlue; color:white" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#Required libraries" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Required libraries<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">1</span></a>
   <a class="list-group-item list-group-item-action" data-toggle="list" href="#I/O" role="tab" aria-controls="profile" style="color:DarkSlateBlue">I/O<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">2</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Custom functions" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Custom functions<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">3</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Data loading" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Data loading<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">4</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#Content based recommender" role="tab" aria-controls="profile" style="color:DarkSlateBlue">Content based recommender<span class="badge badge-primary badge-pill" style="background-color:steelblue; color:white">5</span></a>

<a id='Required libraries'></a>
<h1 style="color:DarkSlateBlue" >Required libraries</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [11]:
#################Libraries##############
#General libraries
import os
import sys

#Data analysis libraries
import pandas as pd
pd.options.display.max_colwidth = 1000
import numpy as np
import operator
import ast
pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

#Visualization libraries
from matplotlib import pyplot as plt
from wordcloud import WordCloud
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode (connected = True)
import ipywidgets as widgets

#Sklearn
from mlxtend.preprocessing import TransactionEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

#Nltk
import nltk
#nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#Image processing
from PIL import Image
import requests
from io import BytesIO

#String similarity
from pyjarowinkler import distance as jaro_distance
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from strsimpy.jaro_winkler import JaroWinkler

#Singular value decomposition
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

############################WARNINGS######################
import warnings
warnings.filterwarnings('ignore')

##################################DISPLAY###################################
from IPython.core.display import display, HTML,clear_output, Markdown
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:90% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

#from itertools import combinations,product
#!pip install strsimpy
#from strsimpy.normalized_levenshtein import NormalizedLevenshtein

<a id='I/O'></a>
<h1 style="color:DarkSlateBlue" >I/O</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [2]:
path_metadata = './dat/movies_metadata.csv'
path_credits = './dat/credits.csv'
path_keywords = './dat/keywords.csv'

<a id='Custom functions'></a>
<h1 style="color:DarkSlateBlue" >Custom functions</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [3]:
########################STEP 1###############################################
#Tokens
#def processing(df_input,column_identifier,column_similarities):
#    tokens = df_input[[column_identifier, column_similarities]].drop_duplicates(column_similarities).reset_index().drop(columns=['index'])
#    print("Number of different titles:",tokens.shape[0])
#    tokens[column_similarities] = tokens[column_similarities].fillna('NA')
#    indices = pd.Series(tokens.index, index=tokens[column_identifier]).drop_duplicates()
#    return tokens, indices

#########################STEP 2###############################################
#Let's create the vectorizer and the tfidf matrix
def tfidf(df_input, column_similarities, column_identifier):
    vectorizer = TfidfVectorizer(stop_words='english',max_features=None) #It gets the features that will make up the sparse matrix
    tfidf_matrix = vectorizer.fit_transform(df_input[column_similarities])
    #column_names = vectorizer.get_feature_names()
    #df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=column_names,index = df_input[column_identifier])
    #n_components = df_tfidf_matrix.shape[1]
    #n_vectors = df_tfidf_matrix.shape[0]
    #print("Tfidf matrix shape:",df_tfidf_matrix.shape)
    #display("Every title is transformed in a vector of {} components, total number of different words. As there are {} distinct titles we have {} distinct vectors".format(n_components,n_vectors,n_vectors),df_tfidf_matrix)
    return tfidf_matrix

########################STEP 3################################################
#Let's create the cosine similarity matrix
def similarities(df_input, tfidf_matrix, column_identifier):
    cosine_sim = cosine_similarity(tfidf_matrix)
    df_cosine_sim = pd.DataFrame(cosine_sim, columns = df_input[column_identifier],index = df_input[column_identifier])
    print("Cosine similarity matrix shape:",df_cosine_sim.shape)
    return df_cosine_sim

In [4]:
def calculate_jaro_distance(*, selected_title, all_possible_titles, num_similarities):
    #Jaro similarity
    similarity_jaro_list = []
    title_names = [title for title in all_possible_titles if str(title)!="nan"]
    for title_name in title_names:
        similarity_jaro = jaro_distance.get_jaro_distance(selected_title.lower(), title_name.lower())
        similarity_jaro_list.append(similarity_jaro)
    titles_dict_jaro = dict(zip(title_names, similarity_jaro_list))
    titles_dict_sorted_jaro = dict(sorted(titles_dict_jaro.items(), key=operator.itemgetter(1),reverse=True)[:num_similarities])
    df_similarities_jaro = pd.DataFrame(list(titles_dict_sorted_jaro.items()),columns = ['title','similarity']) 
    return df_similarities_jaro

In [5]:
def get_most_likely_items_cosine_similarity(*,items,max_number_of_predictions,df_similarity):
    df_transactions_cosine = df_similarity[df_similarity['title'].isin(items)].drop(columns=items)
    display(df_transactions_cosine.head())
    df_most_similar_items = df_transactions_cosine.drop(columns=['title']).sum(axis = 0).reset_index().rename(columns={0:'similarity'}).sort_values(by="similarity",ascending=False)
    fig = px.bar(df_most_similar_items.head(max_number_of_predictions), x="title",y="similarity",title="Recommended movies (using cosine similarities)", 
           labels={'similarity': "Similarityyy"}, height=500)
    fig.show()

<a id='Data loading'></a>
<h1 style="color:DarkSlateBlue" >Data loading</h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

In [28]:
df_metadata = pd.read_csv(path_metadata,low_memory=False)
df_metadata['id'] = df_metadata['id'].astype('str')
df_metadata['revenue'] = df_metadata['revenue'].astype('float')
df_metadata['imdb_id'] = df_metadata['imdb_id'].str.replace('tt','')
df_metadata = df_metadata[df_metadata['revenue']>10_000_000].drop_duplicates('title')
display(df_metadata.head(1))

#Credits dataset
df_credits = pd.read_csv(path_credits)
df_credits['id'] = df_credits['id'].astype('str')

#Keywords dataset
df_keywords = pd.read_csv(path_keywords)
df_keywords['id'] = df_keywords['id'].astype('str')

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [29]:
#Image that we will use in the notebook
#response = requests.get("https://wpamelia.com/wp-content/uploads/2019/06/loading1.jpg")
#image = Image.open(BytesIO(response.content)) 

<a id='Content based recommender'></a>
<h1 style="color:DarkSlateBlue"> Content based recommender </h1> 

<a href="#top" class="btn btn-primary btn-sm" role="button" aria-pressed="true" style="color:white" data-toggle="popover">Go to TOC</a>

Previously, we built a recommender that only took the title into consideration. Now we are gonna build a content based recommender that takes **genre**, **director**, **cast** and **keywords** into consideration.

From now on, in order to reduce the computation cost, we will be analysing just the movies that had **at least 1 million dollars in revenues**.

In [30]:
####################Let's create a column with all the info for the content based recommender############################
#Let's merge the movies dataset with the credits dataset and the keywords dataset
df_metadata1 = df_metadata.merge(df_credits, on='id',how="inner").drop_duplicates()
df_metadata2 = df_metadata1.merge(df_keywords, on='id',how="inner").drop_duplicates().drop_duplicates('title')


print("Number of movies with revenues greater than 10 million dollars:",df_metadata2.shape[0])

#Literal eval
df_metadata2['crew'] = df_metadata2['crew'].apply(ast.literal_eval)
df_metadata2['cast'] = df_metadata2['cast'].apply(ast.literal_eval)
df_metadata2['keywords'] = df_metadata2['keywords'].apply(ast.literal_eval)

#######################GENRES###########################
#Let's get the genres of each movie
df_metadata2['genre_formatted'] = df_metadata2['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


######################DIRECTOR#########################
#Let's get the directors of each movie
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

df_metadata2['director'] = df_metadata2['crew'].apply(get_director) #get diretor from crew column
df_metadata2['director'] = df_metadata2['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", ""))) #lower case and remove white spaces 
df_metadata2['director'] = df_metadata2['director'].apply(lambda x: [x,x]) #mention director twice times to weigh it more



#####################CAST##############################
df_metadata2['cast_formatted'] = df_metadata2['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df_metadata2['cast_formatted'] = df_metadata2['cast_formatted'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
df_metadata2['cast_formatted'] = df_metadata2['cast_formatted'].apply(lambda x: x[:3] if len(x) >=3 else x) #keep first 3 actors from the list



####################KEYWORDS############################
#stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
df_metadata2['keywords_formatted'] = df_metadata2['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
#df_metadata2['keywords_formatted'] = df_metadata2['keywords_formatted'].apply(lambda x: [stemmer.stem(i) for i in x]) #stem of the word
df_metadata2['keywords_formatted'] = df_metadata2['keywords_formatted'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x]) #lemma of the word
df_metadata2['keywords_formatted'] = df_metadata2['keywords_formatted'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x]) #lowercase and without white spaces

##################COMBINE PREVIOUS COLUMNS: GENRE, DIRECTOR, CAST, KEYWORDS#############
df_metadata2['soup'] = df_metadata2['genre_formatted'] + df_metadata2['director'] + df_metadata2['cast_formatted'] + df_metadata2['keywords_formatted']
df_metadata2['soup'] = df_metadata2['soup'].apply(lambda x: ' '.join(x))

df_metadata2.head()

Number of movies with revenues greater than 10 million dollars: 4270


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,genre_formatted,director,cast_formatted,keywords_formatted,soup
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2x...","[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]","[Animation, Comedy, Family]","[johnlasseter, johnlasseter]","[tomhanks, timallen, donrickles]","[jealousy, toy, boy, friendship, friend, rivalry, boynextdoor, newtoy, toycomestolife]",Animation Comedy Family johnlasseter johnlasseter tomhanks timallen donrickles jealousy toy boy friendship friend rivalry boynextdoor newtoy toycomestolife
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {

#Filtering keywords that only appear once
s = df_metadata2.apply(lambda x: pd.Series(x['keywords_formatted']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword_formatted'
s = s.value_counts()

s = s[s > 1]

def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

df_metadata2['keywords_formatted'] = df_metadata2['keywords_formatted'].apply(filter_keywords)

#from nltk.corpus import stopwords
#stop = stopwords.words('english')
#tokens['title_clean'] = tokens['title'].apply(lambda x: x.lower()).apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [31]:
###########Let's call the functions
#tokens_content_based, indices_content_based = processing(df_input = df_metadata2, column_identifier = 'title', column_similarities = 'soup')
tfidf_matrix_content_based= tfidf(df_input = df_metadata2, column_similarities = 'soup', column_identifier = 'title')
cosine_sim_content_based = similarities(df_input = df_metadata2, tfidf_matrix = tfidf_matrix_content_based, column_identifier = 'title').reset_index()
print("Size of matrix:",sys.getsizeof(cosine_sim_content_based)/10**6 ,"MB")
display(cosine_sim_content_based)

Cosine similarity matrix shape: (4270, 4270)
Size of matrix: 146.174817 MB


title,title,Toy Story,Jumanji,Waiting to Exhale,Father of the Bride Part II,Heat,Sudden Death,GoldenEye,The American President,Balto,Nixon,Cutthroat Island,Casino,Sense and Sensibility,Ace Ventura: When Nature Calls,Money Train,Get Shorty,Assassins,Leaving Las Vegas,Now and Then,Dangerous Minds,Twelve Monkeys,Babe,Dead Man Walking,Mortal Kombat,To Die For,How To Make An American Quilt,Se7en,Pocahontas,The Usual Suspects,Home for the Holidays,Mr. Holland's Opus,Friday,From Dusk Till Dawn,Fair Game,Bed of Roses,White Squall,Mary Reilly,Vampire in Brooklyn,Broken Arrow,Happy Gilmore,The Bridges of Madison County,Muppet Treasure Island,Braveheart,Taxi Driver,Rumble in the Bronx,Boomerang,Flirting with Disaster,The Birdcage,Bad Boys,...,The Shack,Power Rangers,Alien: Covenant,CHiPS,Going in Style,RRRrrrr!!!,Smurfs: The Lost Village,The Zookeeper's Wife,Gifted,24,Diary of a Wimpy Kid: The Long Haul,Baywatch,Unforgettable,Snatched,The Fate of the Furious,Captain Underpants: The First Epic Movie,Cars 3,In This Corner of the World,How to Be a Latin Lover,Nasha Russia: Yaytsa sudby,Baahubali 2: The Conclusion,The Irony of Fate. The Sequel,Kabhi Alvida Naa Kehna,The Beguiled,Baby Driver,Rough Night,Weekend Pass,Boj S Tenyu 2: Revansh,Lovey-Dovey 2,Love and the City 2,High Security Vacation,Despicable Me 3,War for the Planet of the Apes,Kidnap,Valerian and the City of a Thousand Planets,Mudhalvan,Fanaa,Atomic Blonde,Dunkirk,Bairavaa,Confidential Assignment,Transformers: The Last Knight,"Mommies, Happy New Year!",Good Time,The Dark Tower,My Old Classmate,The Emoji Movie,Wind River,Baasha,Sivaji: The Boss
0,Toy Story,1.000000,0.010875,0.005908,0.004972,0.000000,0.000000,0.000000,0.005690,0.024766,0.000000,0.000000,0.000000,0.000000,0.005282,0.005687,0.004462,0.035680,0.000000,0.015013,0.000000,0.000000,0.013370,0.000000,0.000000,0.005329,0.000000,0.000000,0.027187,0.000000,0.005437,0.010275,0.006011,0.000000,0.000000,0.000000,0.000000,0.000000,0.006603,0.000000,0.005961,0.000000,0.017065,0.000000,0.000000,0.004720,0.004821,0.004768,0.004826,0.003570,...,0.000000,0.000000,0.000000,0.004862,0.005093,0.006404,0.040040,0.000000,0.000000,0.000000,0.020740,0.005248,0.000000,0.005142,0.000000,0.034818,0.038148,0.019413,0.005285,0.0,0.000000,0.006811,0.000000,0.000000,0.000000,0.005217,0.004956,0.000000,0.006018,0.006035,0.006112,0.041439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004478,0.000000,0.006205,0.000000,0.000000,0.000000,0.035808,0.000000,0.000000,0.005329
1,Jumanji,0.010875,1.000000,0.000000,0.000000,0.000000,0.009606,0.006698,0.000000,0.017142,0.000000,0.009396,0.000000,0.000000,0.009006,0.000000,0.000000,0.007027,0.000000,0.010342,0.000000,0.000000,0.019927,0.000000,0.011826,0.013692,0.000000,0.000000,0.011689,0.000000,0.000000,0.010287,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006117,0.000000,0.000000,0.020832,0.000000,0.000000,0.000000,0.000000,0.000000,0.043208,0.000000,...,0.012252,0.008161,0.000000,0.000000,0.000000,0.000000,0.023549,0.000000,0.000000,0.000000,0.014287,0.000000,0.000000,0.000000,0.000000,0.011555,0.022436,0.000000,0.000000,0.0,0.009580,0.000000,0.000000,0.052612,0.000000,0.000000,0.000000,0.000000,0.015462,0.000000,0.000000,0.024372,0.000000,0.000000,0.005772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010849,0.000000,0.000000,0.014629,0.000000,0.011884,0.000000,0.000000,0.000000
2,Waiting to Exhale,0.005908,0.000000,1.000000,0.005965,0.003651,0.000000,0.000000,0.024787,0.000000,0.005668,0.000000,0.005901,0.034737,0.006336,0.006823,0.026367,0.000000,0.013058,0.010295,0.004241,0.000000,0.009168,0.003371,0.000000,0.011714,0.013856,0.000000,0.009429,0.004937,0.023688,0.004652,0.007212,0.000000,0.019545,0.018267,0.004021,0.016393,0.118877,0.003582,0.007151,0.012511,0.006386,0.004530,0.004673,0.005663,0.016187,0.016009,0.005790,0.004283,...,0.027218,0.000000,0.000000,0.005833,0.011196,0.007683,0.007219,0.005035,0.004351,0.000000,0.007761,0.006296,0.0058

In [32]:
#Let's plot the results
# Button, text box, output
layout = widgets.Layout(width='400px', height='25px') #set width and height
butt_cosine_content = widgets.Button(description='Display similar titles based on cosine similarity',layout = layout,button_style='success')
items_cosine_content = widgets.Text(value='Toy Story',description='Title',layout=layout)
num_similar_movies_cosine_content = widgets.Dropdown(options=list(range(1,21)),value=10,description='Num similar movies',disabled=False,layout=layout)
output_cosine_content = widgets.Output()

def on_butt_clicked(b):
    with output_cosine_content:
        clear_output()
        itemset = list(items_cosine_content.value.split(","))
        itemset = [item.strip().title() for item in itemset]
        print(itemset)
        items_no_reconocidos = [item for item in itemset if item not in list(df_metadata2['title'].unique())]
        if len(items_no_reconocidos)==0:
            #################################ITEM-TO-ITEM COLLABORATIVE FILTERING USING COSINE SIMILARITIES##########################
            get_most_likely_items_cosine_similarity(items = itemset, max_number_of_predictions = num_similar_movies_cosine_content.value, df_similarity = cosine_sim_content_based)
        else:
            if len(items_no_reconocidos)==1:
                print(f"The title {items_no_reconocidos} is not recognised", "\n")
            if len(items_no_reconocidos)>1:
                print(f"The titles {items_no_reconocidos} are not recognised", "\n")
            for item in items_no_reconocidos:
                df_similarities_jaro = calculate_jaro_distance(selected_title = item, all_possible_titles = df_metadata2['title'].unique(), num_similarities = 1)
                print("Maybe you meant:",df_similarities_jaro['title'].values[0])
                
butt_cosine_content.on_click(on_butt_clicked)
widgets.VBox([butt_cosine_content,
              items_cosine_content,
              num_similar_movies_cosine_content,
              output_cosine_content])  